<a href="https://colab.research.google.com/github/Rakesh-Nagaraju/Chatbot-using-RNN-and-NLP/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import numpy as np
#nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import torch
import torch.nn as nn 

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        
        self.l2 = nn.Linear(hidden_size, hidden_size)
        
        self.l3 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [ ]:

import json

import numpy as np 

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


with open('/content/drive/MyDrive/Chatbot/intents.json','r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index] 
    
    def __len__(self):
        return self.n_samples
    
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])

learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True, num_workers = 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

#loss, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words= words.to(device)
    labels= labels.to(device)
    
    outputs = model(words)
    loss = criterion(outputs, labels)

    #backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if (epoch + 1) % 100 == 0:
    print(f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}') 
print(f'final loss , loss={loss.item():.4f}') 

data = {
    "model_state" : model.state_dict(),
    "input_size" : input_size,
    "output_size" : output_size,
    "hidden_size" : hidden_size,
    "all_words" : all_words,
    "tags" : tags
}

File = "/content/drive/MyDrive/Chatbot/data.pth"
torch.save(data, File)
print(f'Training Complete. File save to {File}')

epoch 100/1000, loss=0.6743
epoch 200/1000, loss=0.4504
epoch 300/1000, loss=0.0289
epoch 400/1000, loss=0.0082
epoch 500/1000, loss=0.0052
epoch 600/1000, loss=0.0018
epoch 700/1000, loss=0.0012
epoch 800/1000, loss=0.0012
epoch 900/1000, loss=0.0006
epoch 1000/1000, loss=0.0003
final loss , loss=0.0003
Training Complete. File save to /content/drive/MyDrive/Chatbot/data.pth


In [ ]:
import random
import json
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('/content/drive/MyDrive/Chatbot/intents.json', 'r') as f:
  intents = json.load(f)

File  = "/content/drive/MyDrive/Chatbot/data.pth"
data = torch.load(File)

input_size = data["input_size"]
output_size = data["output_size"]
hidden_size = data["hidden_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Alexa_typo"
print("Let's Chat! type 'quit' to exit")
while True:
  sentence = input("You: ")
  if sentence == "quit":
    break
  sentence = tokenize(sentence)
  x = bag_of_words(sentence, all_words)
  x = x.reshape(1, x.shape[0])
  x = torch.from_numpy(x).to(device)

  output = model(x).to(device)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f"{bot_name}: {random.choice(intent['responses'])} ") 
  else:
    print(f"{bot_name}: I do not Understand!!! ")

Let's Chat! type 'quit' to exit
You: How is the weather today?
Alexa_typo: Hi there, what can I do for you? 
You: tell me a joke
Alexa_typo: What did the buffalo say when his son left for college? Bison. 
You: quit
